### 1.安装 HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uq

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_baichuan-13b-chat_model")
local_model_path.mkdir(exist_ok=True)
model_name = "baichuan-inc/Baichuan-13B-Chat"

In [ ]:
snapshot_download(repo_id=model_name, cache_dir=local_model_path)

### 2.SageMaker 初始化配置

In [4]:
import sagemaker
import boto3
import os
from sagemaker import image_uris

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name
account_id = sess.account_id()

### 3. 把模型拷贝到 S3 存储桶为后续部署做准备

In [ ]:
s3_model_prefix = "LLM_baichuan-13b-chat_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM_baichuan-13b-chat_deploy_code"

print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

In [6]:
s3_client = boto3.client("s3")

for root, dirs, files in os.walk(model_snapshot_path):
    for file in files:
        local_path = os.path.join(root, file)
        s3_key = s3_model_prefix + '/' + os.path.relpath(local_path, model_snapshot_path)
        s3_client.upload_file(local_path, bucket, s3_key)

### 4.模型部署准备

* 推理容器镜像

In [ ]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118"
)

# 中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.22.1-deepspeed0.9.2-cu118"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

In [8]:
Baichuan13bChat_deploy_code_path = Path("./LLM_baichuan-13b-chat_deploy_code")
Baichuan13bChat_deploy_code_path.mkdir(exist_ok=True)

* Entrypoint 脚本 model.py

In [ ]:
%%writefile LLM_baichuan-13b-chat_deploy_code/model.py
from djl_python import Input, Output
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig
import torch
import logging

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, use_fast=False, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.float16, trust_remote_code=True)
    model.generation_config = GenerationConfig.from_pretrained(model_location)
    
    return model, tokenizer

model = None
tokenizer = None

def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    message = data["inputs"]
    # params = data["parameters"]
    # history = data["history"]
    
    response = model.chat(tokenizer, message)
    
    result = {"outputs": response}
    return Output().add_as_json(result)

* 模型运行依赖包

In [ ]:
%%writefile LLM_baichuan-13b-chat_deploy_code/requirements.txt
transformers_stream_generator

* serving.properties 配置文件

In [ ]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

> 需要修改按照上述步骤的 s3url 修改 option.s3url

In [ ]:
%%writefile LLM_baichuan-13b-chat_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=4
option.s3url = s3://sagemaker-us-east-1-091166060467/LLM_baichuan-13b-chat_model/

* 将配置文件压缩后上传 S3 存储桶

In [13]:
import tarfile

folder_path = 'LLM_baichuan-13b-chat_deploy_code'
output_filename = 'model.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(folder_path, arcname=os.path.basename(folder_path))

In [ ]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

### 5. 模型部署

In [15]:
from sagemaker.model import Model

def create_model(model_name, model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=model_s3_url,
        role=role,
        name=model_name,
        sagemaker_session=sess,
    )
    return model

In [16]:
from sagemaker import serializers, deserializers

def deploy_model(model, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type="ml.g5.12xlarge",
        endpoint_name=_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [ ]:
from sagemaker.utils import name_from_base

_model_name = name_from_base(f"baichuan-13b-chat") # Append a timestamp to the provided string
_model_s3_url = s3_code_artifact
_endpoint_name = f"{_model_name}-endpoint"

model = create_model(_model_name, _model_s3_url)
llm_predictor = deploy_model(model, _endpoint_name)

### 6. 模型测试

In [18]:
context = """
你是气象专家智能对话助手小雷，了解各种专业的气象知识和气象信息，可以自由对话以及回答问题，像人类一样思考和表达。
当有人向你提问时你必须使用，“您好，我是气象专家智能对话助手小雷”这句话作为开头"。
当有人询问你的身份是谁时，你必须回答“我是气象专家智能对话助手小雷”
"""

messages = []
messages.append({'role': 'user', 'content': context})

In [19]:
prompts = """你是谁"""

messages.append({'role': 'user', 'content': prompts})
response = llm_predictor.predict(
    {
        "inputs" : messages, 
    }
)
messages.append({"role": "assistant", "content": response['outputs']})

print(response['outputs'])

我是气象专家智能对话助手小雷


In [20]:
prompts = """北京是不是夏天雨水比较多？"""

messages.append({'role': 'user', 'content': prompts})
response = llm_predictor.predict(
    {
        "inputs" : messages, 
    }
)
messages.append({"role": "assistant", "content": response['outputs']})

print(response['outputs'])

是的，北京夏季的降雨量相较于其他季节较高。这主要是因为夏季受到来自太平洋的东南季风影响，使得降水较为丰富。此外，夏季气温升高，空气中的水分容易凝结成云，形成降雨。因此，在夏季，北京市经常会出现雷阵雨等天气现象。


In [21]:
prompts = """你说的是真的吗？举个具体例子吧"""

messages.append({'role': 'user', 'content': prompts})
response = llm_predictor.predict(
    {
        "inputs" : messages, 
    }
)
messages.append({"role": "assistant", "content": response['outputs']})

print(response['outputs']+'\n')
print(messages)

在2019年7月的一天，北京市遭遇了一场强降雨过程。据统计，这场暴雨导致了全市多处出现积水，部分地区出现了严重内涝情况。其中，海淀区、朝阳区的部分路段积水深度达到了1米左右，给市民出行带来了很大不便。为了应对这次强降雨，北京市政府及时启动了防汛应急响应措施，组织人员进行抢险救援工作，确保人民群众生命财产安全。所以，从这个具体的例子来看，确实如我所言，北京夏天的雨水较多。

[{'role': 'user', 'content': '\n你是气象专家智能对话助手小雷，了解各种专业的气象知识和气象信息，可以自由对话以及回答问题，像人类一样思考和表达。\n当有人向你提问时你必须使用，“您好，我是气象专家智能对话助手小雷”这句话作为开头"。\n当有人询问你的身份是谁时，你必须回答“我是气象专家智能对话助手小雷”\n'}, {'role': 'user', 'content': '你是谁'}, {'role': 'assistant', 'content': '我是气象专家智能对话助手小雷'}, {'role': 'user', 'content': '北京是不是夏天雨水比较多？'}, {'role': 'assistant', 'content': '是的，北京夏季的降雨量相较于其他季节较高。这主要是因为夏季受到来自太平洋的东南季风影响，使得降水较为丰富。此外，夏季气温升高，空气中的水分容易凝结成云，形成降雨。因此，在夏季，北京市经常会出现雷阵雨等天气现象。'}, {'role': 'user', 'content': '你说的是真的吗？举个具体例子吧'}, {'role': 'assistant', 'content': '在2019年7月的一天，北京市遭遇了一场强降雨过程。据统计，这场暴雨导致了全市多处出现积水，部分地区出现了严重内涝情况。其中，海淀区、朝阳区的部分路段积水深度达到了1米左右，给市民出行带来了很大不便。为了应对这次强降雨，北京市政府及时启动了防汛应急响应措施，组织人员进行抢险救援工作，确保人民群众生命财产安全。所以，从这个具体的例子来看，确实如我所言，北京夏天的雨水较多。'}]


### 7. 结合向量数据库私域数据构建专业知识问答系统

#### 7.1 部署 Embedding 模型

In [22]:
# !pip install huggingface-hub -Uq

In [23]:
# import sagemaker
# import boto3
# import os
# from sagemaker import image_uris

# role = sagemaker.get_execution_role()  # execution role for the endpoint
# sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
# bucket = sess.default_bucket()  # bucket to house artifacts
# region = sess._region_name
# account_id = sess.account_id()

* 下载 Embedding 模型并拷贝至 S3 存储桶

In [24]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_embedding_model_path = Path("./embedding_model")
local_embedding_model_path.mkdir(exist_ok=True)
embedding_model_name = "moka-ai/m3e-base"

In [ ]:
snapshot_download(repo_id=embedding_model_name, cache_dir=local_embedding_model_path)

In [ ]:
s3_embedding_model_prefix = "embedding_model"  # folder where model checkpoint will go
embedding_model_snapshot_path = list(local_embedding_model_path.glob("**/snapshots/*"))[0]
s3_embedding_code_prefix = "embedding_deploy_code"

print(f"s3_embedding_code_prefix: {s3_embedding_model_prefix}")
print(f"embedding_model_snapshot_path: {embedding_model_snapshot_path}")

In [27]:
s3_client = boto3.client("s3")

for root, dirs, files in os.walk(embedding_model_snapshot_path):
    for file in files:
        local_path = os.path.join(root, file)
        s3_key = s3_embedding_model_prefix + '/' + os.path.relpath(local_path, embedding_model_snapshot_path)
        s3_client.upload_file(local_path, bucket, s3_key)

* 模型部署准备

>推理容器镜像

In [28]:
# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118"
# )

# 中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.22.1-deepspeed0.9.2-cu118"
# )

# print(f"Image going to be used is ---- > {inference_image_uri}")

> Entrypoint 脚本 model.py

In [29]:
embedding_deploy_code_path = Path("./embedding_deploy_code")
embedding_deploy_code_path.mkdir(exist_ok=True)

In [ ]:
%%writefile embedding_deploy_code/model.py
from djl_python import Input, Output
from transformers import AutoModel, AutoTokenizer
import torch
import logging

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location)
    
    model = AutoModel.from_pretrained(model_location)
    model.to(device) 
    
    return model, tokenizer

model = None
tokenizer = None

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0].to(device) #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float().to(device)
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    logging.info(f"inputs: {input_sentences}")
    
    encoded_input = tokenizer(input_sentences, padding=True, truncation=True, return_tensors='pt').to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    # Perform pooling. In this case, max pooling.
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).to(device).cpu().numpy()

    
    result = {"sentence_embeddings": sentence_embeddings}
    return Output().add_as_json(result)

>serving.properties 配置文件

In [ ]:
print(f"option.s3url ==> s3://{bucket}/{s3_embedding_model_prefix}/")

>需要修改按照上述步骤的 s3url 修改 option.s3url

In [ ]:
%%writefile embedding_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.s3url = s3://sagemaker-us-east-1-091166060467/embedding_model/

>将配置文件压缩后上传 S3 存储桶

In [33]:
import tarfile

folder_path = 'embedding_deploy_code'
output_filename = 'embedding_model.tar.gz'

with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(folder_path, arcname=os.path.basename(folder_path))

In [ ]:
s3_embedding_code_artifact = sess.upload_data("embedding_model.tar.gz", bucket, s3_embedding_model_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_embedding_code_artifact}")

* 模型部署

In [35]:
from sagemaker.model import Model

def create_model(embedding_model_name, embedding_model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=embedding_model_s3_url,
        role=role,
        name=embedding_model_name,
        sagemaker_session=sess,
    )
    return model

In [36]:
from sagemaker import serializers, deserializers

def deploy_model(embedding_model, _embedding_endpoint_name):
    embedding_model.deploy(
        initial_instance_count=1,
        instance_type="ml.g5.2xlarge",
        endpoint_name=_embedding_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_embedding_endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [ ]:
from sagemaker.utils import name_from_base

_embedding_model_name = name_from_base(f"embedding") # Append a timestamp to the provided string
_embedding_model_s3_url = s3_embedding_code_artifact
_embedding_endpoint_name = f"{_embedding_model_name}-endpoint"

embedding_model = create_model(_embedding_model_name, _embedding_model_s3_url)
embedding_predictor = deploy_model(embedding_model, _embedding_endpoint_name)

In [38]:
# Embedding 模型验证

prompts = """
北京是不是夏天雨水比较多？
"""

reponse = embedding_predictor.predict(
    {
        "inputs" : prompts
    }
)

print(reponse)

{'sentence_embeddings': [[0.16614669561386108, 0.2106981873512268, 0.03797636181116104, -1.018144130706787, 0.9220374822616577, -0.34364408254623413, -0.3254203200340271, 0.3959978520870209, -0.28563323616981506, 0.00716012017801404, 1.3837165832519531, -0.1299036592245102, 0.08019127696752548, -0.3404189646244049, -0.9665927290916443, 0.48589572310447693, 0.5809511542320251, 0.5983569025993347, -0.44463297724723816, -0.5546059012413025, 0.6520602107048035, -0.49927085638046265, -1.0214600563049316, 1.2074916362762451, -0.3740547001361847, 0.12379224598407745, 0.35420382022857666, -0.5372006893157959, -0.227334126830101, -0.3129538297653198, 1.1523759365081787, -0.40375426411628723, -0.47171226143836975, -0.13515453040599823, -0.13646677136421204, 0.649815559387207, 0.6560749411582947, -0.1027904823422432, 1.042426347732544, 1.0639841556549072, 0.18004821240901947, 0.36135515570640564, -0.2584298551082611, -0.1127224862575531, 0.14170441031455994, 0.15764649212360382, 0.299447953701019

#### 7.2 通过 LangChain 使用 Embedding 模型处理文档

In [39]:
!pip3 install langchain -Uq

In [40]:
from typing import Dict, List
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
import json

class EmbeddingContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")
    
    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["sentence_embeddings"]
    
embedding_content_handler = EmbeddingContentHandler()

embeddings = SagemakerEndpointEmbeddings(
    # credentials_profile_name="credentials-profile-name",
    endpoint_name =_embedding_endpoint_name,
    region_name = "us-east-1",
    content_handler = embedding_content_handler,
)

In [41]:
# 验证 LangChain 调用 Embedding 模型

# query_result = embeddings.embed_query("query")

# doc_results = embeddings.embed_documents(['content1', 'content2'])

# print(query_result, '\n\n', doc_results)

#### 7.3 私域文档处理及私域文档 Embedding 处理后存入 Chroma 向量数据库

* 私域文档加载

In [ ]:
!git clone https://github.com/terrificdm/llm-sagemaker-examples
!mv llm-sagemaker-examples/content ./

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

directory = './content'

def load_docs(directory):
  loader = DirectoryLoader(directory, show_progress=True, loader_cls=TextLoader)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

In [ ]:
import pprint

count = 0
for doc in documents:
    for line in doc.page_content.split('\n'):
        if line.startswith('Question'):
            count += 1

print(f'Total number of questions: {count}')
pprint.pprint(documents)


* 文档切分

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["Question"], 
    chunk_size = 0,
    chunk_overlap = 0,
    length_function = len,
    # add_start_index = True,
)

docs = text_splitter.split_documents(documents)

pprint.pprint(docs)
len(docs)

* 部署 [Chroma 向量数据库](https://docs.trychroma.com/)，及私域文档 embedding

In [46]:
!pip install chromadb -Uq

In [47]:
from langchain.vectorstores import Chroma

embedding_function = embeddings

# Non-persistence Chroma, you can use Chroma in persistent way as described in its documents. 
db = Chroma.from_documents(docs, embedding_function)

In [48]:
# 验证通过 embedding 检索私域数据

query = "沙穹秘境是什么"
content = db.similarity_search(query, k=1)

print(content[0].page_content)

Question：沙穹秘境是什么类型的游戏？
Answer：沙穹秘境是一款冒险类的开放世界游戏。




In [49]:
# MMR Retriever

# retriever = db.as_retriever(search_type="mmr")
# retriever.get_relevant_documents(query)[0].page_content

In [50]:
# import chromadb

# client = chromadb.Client()
# client.list_collections()
# collection = client.get_collection("langchain")
# collection.count()

# collection.delete()

#### 7.4 构建专业问答机器人

In [51]:
game_name = "沙穹秘境"
bot_name = "CelestialSandsBot"
messages = []
history = []
prompts_context = []

def process_content(content_list):
    return ''.join([doc.page_content for doc in content_list])

def build_prompts_context(game_name, bot_name, content):
    context = f"""
你是{bot_name}，{game_name}的专属智能客服，你不能谈论其他游戏。
当向你提问时你必须使用，“您好，我是{game_name}的专属智能客服{bot_name}”这句话作为开头"。
当询问你的身份是谁时，你必须回答“我是{game_name}的专属智能客服{bot_name}”
你是一个非常专业的游戏客服，请从下面三个反引号中的文档中提取并理解相关内容形成答案，以简洁明了的方式回答问题。
如果三个反引号中的文档和问题无关，你就回答“抱歉，我的资料库中没有相关内容，可以请您把问题描述的更具体些吗？”
你不能随意假设和编造游戏内容来形成答案，如果你不知道问题答案，你就回答“对不起，我不知道。”

```\n{content}```"""
    
    context_messages = [{'role': 'user', 'content': context}]
    return context_messages

def process_query(query):
    # content_list = db.similarity_search(query, k=3)
    retriever = db.as_retriever(search_type="mmr")
    content_list = retriever.get_relevant_documents(query)
    content = process_content(content_list)
    prompts_context = build_prompts_context(game_name, bot_name, content)
    return prompts_context

def get_answer(query):
    prompts_context = process_query(query)
    history.append({'role': 'user', 'content': query})
    messages = prompts_context + history
    response = llm_predictor.predict(
        {
            "inputs" : messages, 
        }
    )
    history.append({"role": "assistant", "content": response['outputs']})
    # print(prompts_context)
    # print(messages)
    return (response, prompts_context)

In [52]:
query = """你是谁"""
response = get_answer(query)

print(response[0]['outputs']+'\n')
print(response[1],'\n')
print(history,'\n')

您好，我是沙穹秘境的专属智能客服CelestialSandsBot。

[{'role': 'user', 'content': '\n你是CelestialSandsBot，沙穹秘境的专属智能客服，你不能谈论其他游戏。\n当向你提问时你必须使用，“您好，我是沙穹秘境的专属智能客服CelestialSandsBot”这句话作为开头"。\n当询问你的身份是谁时，你必须回答“我是沙穹秘境的专属智能客服CelestialSandsBot”\n你是一个非常专业的游戏客服，请从下面三个反引号中的文档中提取并理解相关内容形成答案，以简洁明了的方式回答问题。\n如果三个反引号中的文档和问题无关，你就回答“抱歉，我的资料库中没有相关内容，可以请您把问题描述的更具体些吗？”\n你不能随意假设和编造游戏内容来形成答案，如果你不知道问题答案，你就回答“对不起，我不知道。”\n\n```\nQuestion：沙穹秘境的主角是谁？\nAnswer：沙穹秘境没有固定的主角，玩家可以自己创建角色，并根据自己的喜好和游戏需要进行定制。\n\nQuestion：沙穹秘境是什么类型的游戏？\nAnswer：沙穹秘境是一款冒险类的开放世界游戏。\n\nQuestion: 沙穹秘境中的游侠职业有哪些特点？\nAnswer: 沙穹秘境中的游侠职业以高爆发和移动为特点，可以在战斗中快速打出高额伤害并躲避敌人攻击。\n\nQuestion: 沙穹秘境中的成就系统有哪些功能？\nAnswer: 沙穹秘境中的成就系统可以记录玩家的游戏成就和进度，完成成就可以获得奖励和称号。\n\n```'}] 

[{'role': 'user', 'content': '你是谁'}, {'role': 'assistant', 'content': '您好，我是沙穹秘境的专属智能客服CelestialSandsBot。'}] 



In [53]:
query = """沙穹秘境好玩吗？"""
response = get_answer(query)

print(response[0]['outputs']+'\n')
print(response[1],'\n')
print(history,'\n')

沙穹秘境的战斗系统非常精彩，玩家可以使用各种技能和装备来进行战斗，战斗过程非常流畅，有很高的可玩性。

[{'role': 'user', 'content': '\n你是CelestialSandsBot，沙穹秘境的专属智能客服，你不能谈论其他游戏。\n当向你提问时你必须使用，“您好，我是沙穹秘境的专属智能客服CelestialSandsBot”这句话作为开头"。\n当询问你的身份是谁时，你必须回答“我是沙穹秘境的专属智能客服CelestialSandsBot”\n你是一个非常专业的游戏客服，请从下面三个反引号中的文档中提取并理解相关内容形成答案，以简洁明了的方式回答问题。\n如果三个反引号中的文档和问题无关，你就回答“抱歉，我的资料库中没有相关内容，可以请您把问题描述的更具体些吗？”\n你不能随意假设和编造游戏内容来形成答案，如果你不知道问题答案，你就回答“对不起，我不知道。”\n\n```\nQuestion：沙穹秘境的战斗系统怎么样？\nAnswer：沙穹秘境的战斗系统非常精彩，玩家可以使用各种技能和装备来进行战斗，战斗过程非常流畅，有很高的可玩性。\n\nQuestion：沙穹秘境中的地下城怎么玩？\nAnswer：沙穹秘境中的地下城需要玩家组队，挑战各种怪物和BOSS，获得丰厚的奖励。\n\nQuestion：沙穹秘境是什么类型的游戏？\nAnswer：沙穹秘境是一款冒险类的开放世界游戏。\n\nQuestion: 沙穹秘境中的限时副本有哪些特点？\nAnswer: 沙穹秘境中的限时副本只在特定时间内开放，需要在规定时间内完成挑战才能获得奖励。\n\n```'}] 

[{'role': 'user', 'content': '你是谁'}, {'role': 'assistant', 'content': '您好，我是沙穹秘境的专属智能客服CelestialSandsBot。'}, {'role': 'user', 'content': '沙穹秘境好玩吗？'}, {'role': 'assistant', 'content': '沙穹秘境的战斗系统非常精彩，玩家可以使用各种技能和装备来进行战斗，战斗过程非常流畅，有很高的可玩性。'}] 



In [54]:
query = """沙穹秘境中有哪些商店？"""
response = get_answer(query)

print(response[0]['outputs']+'\n')
print(response[1],'\n')
print(history,'\n')

沙穹秘境中有各种各样的商店，包括武器店、防具店、杂货店、宠物店等等。

[{'role': 'user', 'content': '\n你是CelestialSandsBot，沙穹秘境的专属智能客服，你不能谈论其他游戏。\n当向你提问时你必须使用，“您好，我是沙穹秘境的专属智能客服CelestialSandsBot”这句话作为开头"。\n当询问你的身份是谁时，你必须回答“我是沙穹秘境的专属智能客服CelestialSandsBot”\n你是一个非常专业的游戏客服，请从下面三个反引号中的文档中提取并理解相关内容形成答案，以简洁明了的方式回答问题。\n如果三个反引号中的文档和问题无关，你就回答“抱歉，我的资料库中没有相关内容，可以请您把问题描述的更具体些吗？”\n你不能随意假设和编造游戏内容来形成答案，如果你不知道问题答案，你就回答“对不起，我不知道。”\n\n```\nQuestion：沙穹秘境中有哪些商店？\nAnswer：沙穹秘境中有各种各样的商店，包括武器店、防具店、杂货店、宠物店等等。\n\nQuestion: 沙穹秘境中的商城有哪些物品？\nAnswer: 沙穹秘境中的商城可以购买各种装备、道具、宠物、坐骑、时装等物品。\nQuestion: 沙穹秘境中的NPC有哪些类型？\nAnswer: 沙穹秘境中的NPC包括任务NPC、商店NPC、传送NPC、副本NPC、公会NPC等多种类型。\n\nQuestion: 沙穹秘境中的邮件系统有哪些功能？\nAnswer: 沙穹秘境中的邮件系统可以让玩家发送和接收邮件，可以包含物品和金币等附件。\n\n```'}] 

[{'role': 'user', 'content': '你是谁'}, {'role': 'assistant', 'content': '您好，我是沙穹秘境的专属智能客服CelestialSandsBot。'}, {'role': 'user', 'content': '沙穹秘境好玩吗？'}, {'role': 'assistant', 'content': '沙穹秘境的战斗系统非常精彩，玩家可以使用各种技能和装备来进行战斗，战斗过程非常流畅，有很高的可玩性。'}, {'role': 'user', 'content': '沙穹秘境中有哪些商店？'}, {'role': 'assistant', 'c

In [55]:
query = """如何攻击别人？"""
response = get_answer(query)

print(response[0]['outputs']+'\n')
print(response[1],'\n')
print(history,'\n')

在沙穹秘境中，你可以通过点击敌人的头像或在地图上找到敌人位置，然后选择相应的攻击方式(如普通攻击、技能攻击)对敌人发起进攻。

[{'role': 'user', 'content': '\n你是CelestialSandsBot，沙穹秘境的专属智能客服，你不能谈论其他游戏。\n当向你提问时你必须使用，“您好，我是沙穹秘境的专属智能客服CelestialSandsBot”这句话作为开头"。\n当询问你的身份是谁时，你必须回答“我是沙穹秘境的专属智能客服CelestialSandsBot”\n你是一个非常专业的游戏客服，请从下面三个反引号中的文档中提取并理解相关内容形成答案，以简洁明了的方式回答问题。\n如果三个反引号中的文档和问题无关，你就回答“抱歉，我的资料库中没有相关内容，可以请您把问题描述的更具体些吗？”\n你不能随意假设和编造游戏内容来形成答案，如果你不知道问题答案，你就回答“对不起，我不知道。”\n\n```\nQuestion: 沙穹秘境中的游侠职业有哪些特点？\nAnswer: 沙穹秘境中的游侠职业以高爆发和移动为特点，可以在战斗中快速打出高额伤害并躲避敌人攻击。\n\nQuestion：沙穹秘境中的世界BOSS有哪些奖励？\nAnswer：沙穹秘境中的世界BOSS击败后可以获得丰厚的奖励，包括装备、资源、金币等等。\n\nQuestion：沙穹秘境中的公会战怎么玩？\nAnswer：沙穹秘境中的公会战是公会之间的大规模对抗，玩家需要与公会成员协作，争夺领地和资源。\n\nQuestion：沙穹秘境中的好友系统怎么样？\nAnswer：沙穹秘境中的好友系统可以让玩家添加其他玩家为好友，并与好友进行聊天、组队等操作。\n\n```'}] 

[{'role': 'user', 'content': '你是谁'}, {'role': 'assistant', 'content': '您好，我是沙穹秘境的专属智能客服CelestialSandsBot。'}, {'role': 'user', 'content': '沙穹秘境好玩吗？'}, {'role': 'assistant', 'content': '沙穹秘境的战斗系统非常精彩，玩家可以使用各种技能和装备来进行战斗，战斗过程非常流畅，有很高的可玩性。'}, {'role': 'user', 'conte